In [142]:
import pandas as pd 
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder 
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split

In [118]:
df = pd.read_csv(".\data\practive pyspark.csv")
df.drop(columns ={'flight_date','booking_date','Code'}, inplace =True)
df.head()

,area,gender,age,age_group,year_of_school,degree,job,know_english,know_france,know_chinese,income,flight_status,professionally_staff,customer_service,diversity_product,good_price,easily_transaction,goodlooking_staff,diversity_flighttime,good_construction
0,central,female,69,middle,16,master,manager,1,1,1,33250,1,2,2,1,1,2,2,1,1
1,southern,female,50,middle,12,highshool,officer,0,0,0,6960,1,3,3,3,2,3,3,2,3
2,northern,male,73,elder,12,highshool,officer,1,0,0,11100,1,2,2,1,2,3,5,1,1
3,northern,female,73,elder,12,highshool,officer,0,0,0,11100,1,5,3,2,4,5,3,2,2
4,central,male,69,middle,16,master,officer,1,0,0,16140,0,3,3,3,3,3,3,3,3


In [119]:
#Check missing values
df.isna().sum()

area                    0
gender                  0
age                     0
age_group               0
year_of_school          0
degree                  0
job                     0
know_english            0
know_france             0
know_chinese            0
income                  0
flight_status           0
professionally_staff    0
customer_service        0
diversity_product       0
good_price              0
easily_transaction      0
goodlooking_staff       0
diversity_flighttime    0
good_construction       0
dtype: int64

In [194]:
X = df[['area', 'gender', 'age', 'age_group', 'year_of_school', 'degree', 'job',
       'know_english', 'know_france', 'know_chinese',
       'flight_status', 'professionally_staff', 'customer_service ',
       'diversity_product', 'good_price', 'easily_transaction',
       'goodlooking_staff', 'diversity_flighttime', 'good_construction']]
y = df['income']

In [195]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [196]:
ohe = OneHotEncoder()
imp_ohe = make_pipeline( ohe)
# Making a transformer
ct = make_column_transformer(
    (imp_ohe, ['area', 'gender','age_group','degree','job']),
    remainder='passthrough')

In [206]:
ct.fit_transform(df)

array([[1., 0., 0., ..., 2., 1., 1.],
       [0., 0., 1., ..., 3., 2., 3.],
       [0., 1., 0., ..., 5., 1., 1.],
       ...,
       [0., 0., 1., ..., 5., 2., 2.],
       [0., 0., 1., ..., 2., 2., 2.],
       [0., 1., 0., ..., 1., 3., 3.]])

In [198]:
logreg = LinearRegression()

In [199]:
# Creating a pipeline
pipe = make_pipeline(ct, logreg)

In [200]:
# Fit data using the pipeline
pipe.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('pipeline',
                                                  Pipeline(steps=[('onehotencoder',
                                                                   OneHotEncoder())]),
                                                  ['area', 'gender',
                                                   'age_group', 'degree',
                                                   'job'])])),
                ('linearregression', LinearRegression())])

In [201]:
#predict income
y_pred = pipe.predict(X_test)
y_pred

array([11858.50491347, 17606.62259827, 10150.20347291, 20840.95996622,
       10095.99632822, 11698.41255524, 12807.43731498, 26128.37405615,
        9142.55941702, 14352.36747306,  6674.36453568, 14466.46322206,
       13177.69056404, 11144.47584167, 10490.51830133, 11707.54363269,
        9601.06148428, 12357.80112558, 10615.49429938, 16322.32743837,
        9017.23717064,  8242.18504721, 14678.33384517,  6941.46517573,
        9974.07105779,  9787.78465516,  8252.91683787, 12249.08828709,
        7489.7083966 , 20987.35148728, 28946.99947767, 13506.23985072,
       10849.03702024, 15101.90559345, 12309.61887393, 17883.78797407,
       11853.94313138,  7499.87056736, 27259.06563446, 11477.13491241,
       14121.6917063 , 11658.11855781, 10618.48645197,  9838.21032451,
        8599.17296716,  8354.82500979, 10369.06208247, 26396.61843929,
        5123.32508061, 15799.34731865, 15291.98080042, 12471.7938894 ,
       14213.38030711, 19430.19003894, 12563.88906331, 28867.07764396,
      

In [163]:
import statsmodels.formula.api as smf

In [202]:
mod = smf.ols(formula="income ~ area + gender  + year_of_school+degree+job+ know_france +know_chinese ", data=df)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                 income   R-squared:                       0.775
Model:                            OLS   Adj. R-squared:                  0.771
Method:                 Least Squares   F-statistic:                     168.9
Date:                Wed, 11 Aug 2021   Prob (F-statistic):          1.39e-151
Time:                        17:03:13   Log-Likelihood:                -4772.6
No. Observations:                 500   AIC:                             9567.
Df Residuals:                     489   BIC:                             9614.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept            1.312e+04   2